In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

plt.rcParams['figure.figsize']=[12,8]
matplotlib.rc('font',family='NanumGothic')

df_do=pd.read_excel("./중화면용기.xlsx",encoding="UTF-8")


/home/piai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
df_do=df_do[['주문일시','수량']]

In [3]:
df_do['주문일시']=df_do['주문일시'].astype(str)
type(df_do['주문일시'][0])

str

In [4]:
df_do['주문일시']=df_do['주문일시'].str.slice(start=0,stop=7)
df_do

,주문일시,수량
0,2018-12,1
1,2018-12,1
2,2018-12,4
3,2018-12,1
4,2018-12,1
...,...,...
4991,2018-12,4
4992,2018-12,1
4993,2018-12,1
4994,2018-12,1


In [5]:
df_do['주문일시']=pd.to_datetime(df_do['주문일시'])

In [6]:
df_day=df_do.set_index("주문일시")

In [7]:
df_day=df_do.groupby(by=['주문일시']).sum().groupby(level=[0]).cumsum()

df_day=df_day[:-1]
df_day

,수량
주문일시,
2018-03-01,2
2018-04-01,118
2018-05-01,146
2018-06-01,127
2018-07-01,214
2018-08-01,305
2018-09-01,291
2018-10-01,380
2018-11-01,358


In [8]:
#Train 데이터 이용, AR(1),I(2,차분),MA(0)인 ARIMA 모델
model=ARIMA(df_day,order=(1,2,0))
#데이터 적합
#trend:상수 포함 여부"nc"이면 상수 미포함,full_output:모든 출력결과 표시
result=model.fit(trend="nc",full_output=True)
print(result.summary())

                             ARIMA Model Results                              
Dep. Variable:                  D2.수량   No. Observations:                   13
Model:                 ARIMA(1, 2, 0)   Log Likelihood                 -91.677
Method:                       css-mle   S.D. of innovations            272.706
Date:                Thu, 23 Jul 2020   AIC                            187.353
Time:                        11:42:39   BIC                            188.483
Sample:                    05-01-2018   HQIC                           187.121
                         - 05-01-2019                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
ar.L1.D2.수량    -0.6884      0.297     -2.321      0.020      -1.270      -0.107
                                    Roots                                    
                  Real          Imaginary         

/home/piai/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/home/piai/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


In [9]:
fig=result.predict(start="2018-12",end="2019-07",typ='levels')
fig

2018-12-01    412.415039 
2019-01-01    590.742584 
2019-02-01    1085.012300
2019-03-01    913.399070 
2019-04-01    775.677313 
2019-05-01    1488.167442
2019-06-01    982.349871 
2019-07-01    712.686634 
Freq: MS, dtype: float64

In [10]:
#전체 기간의 실제값, 예측값 확인
y_pred=result.predict(start="2018-05",end="2019-09",typ='levels')
y_pred
df_merge=df_day # 실제값 저장
df_merge["예측값"]=y_pred.astype(int)#예측값 저장 및 변수명 지정
df_merge["차이"]=df_merge["수량"]-df_merge["예측값"]

#추세그래프 생성 및 확인
df_merge

,수량,예측값,차이
주문일시,,,
2018-03-01,2,NaN,NaN
2018-04-01,118,NaN,NaN
2018-05-01,146,234.0,-88.0
2018-06-01,127,234.0,-107.0
2018-07-01,214,140.0,74.0
2018-08-01,305,228.0,77.0
2018-09-01,291,393.0,-102.0
2018-10-01,380,349.0,31.0
2018-11-01,358,398.0,-40.0


In [11]:
df_quarter=df_merge[-5:-2]
df_half_year=df_merge[-6:]

In [12]:
df_half_year

,수량,예측값,차이
주문일시,,,
2018-12-01,547,412.0,135.0
2019-01-01,858,590.0,268.0
2019-02-01,737,1085.0,-348.0
2019-03-01,830,913.0,-83.0
2019-04-01,1283,775.0,508.0
2019-05-01,816,1488.0,-672.0


In [13]:
df_half_year.loc['Total',:]= df_half_year.sum(axis=0)
df_quarter.loc['Total',:]= df_quarter.sum(axis=0)

/home/piai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/piai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_half_year

,수량,예측값,차이
주문일시,,,
2018-12-01 00:00:00,547.0,412.0,135.0
2019-01-01 00:00:00,858.0,590.0,268.0
2019-02-01 00:00:00,737.0,1085.0,-348.0
2019-03-01 00:00:00,830.0,913.0,-83.0
2019-04-01 00:00:00,1283.0,775.0,508.0
2019-05-01 00:00:00,816.0,1488.0,-672.0
Total,5071.0,5263.0,-192.0


In [15]:
df_quarter

,수량,예측값,차이
주문일시,,,
2019-01-01 00:00:00,858.0,590.0,268.0
2019-02-01 00:00:00,737.0,1085.0,-348.0
2019-03-01 00:00:00,830.0,913.0,-83.0
Total,2425.0,2588.0,-163.0
